In [1]:
import pandas as pd
from pathlib import Path
import os

In [2]:
csv_fns = list(Path('.').glob('*.csv'))

In [3]:
csv_fns

[PosixPath('revived-donkey-11.csv'),
 PosixPath('t5large-both-0-few-shot-revision.csv')]

In [4]:
revised   = pd.read_csv(csv_fns[1])
unrevised = pd.read_csv(csv_fns[0])

revised = revised.drop('input_(counterfactual)explain',axis=1)
revised = revised.drop('answers_labels',axis=1)
unrevised = unrevised.drop('input_(counterfactual)explain',axis=1)

In [5]:
merged = pd.merge(unrevised,revised,how='left',validate='1:1',on=['input_predict'],suffixes=['_unrevised','_revised'])

In [6]:
rename = {
    "input_predict": "Prompt",
    "answers_labels": "Ground truth",
    "rationales_labels_unrevised": "Unrevised rationale",
    "rationales_labels_revised": "Revised rationale",
    "answers_preds_unrevised": "Answers (student MT+CF:Unrevised)",
    "answers_preds_revised": "Answers (student MT+CF:Revised)",
    "rationales_preds_unrevised": "Explanation (student MT+CF:Unrevised)",
    "rationales_preds_revised": "Explanation (student MT+CF:Revised)",
    }

In [7]:
merged = merged.rename(rename, axis=1)
merged = merged[list(rename.values())]

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
merged.head()

,Prompt,Ground truth,Unrevised rationale,Revised rationale,Answers (student MT+CF:Unrevised),Answers (student MT+CF:Revised),Explanation (student MT+CF:Unrevised),Explanation (student MT+CF:Revised)
0,"[predict] Question: A beaver is know for building prowess, their supplies come from where? Choices: (a) british columbia (b) body of water (c) wooded area (d) pay debts (e) zoo",The answer is wooded area,"Beavers rely on the wooded areas to obtain materials for building their dams and lodges. This provides a convenient and abundant source of raw materials, which is essential for the beavers' construction activities.","Beavers rely on these areas for the necessary materials to build their dams and lodges. This choice is better than the other options because beavers require specific types of wood and other materials found in wooded areas to construct their homes effectively. For example, willows and cottonwoods are ideal for building dams, while aspen, birch, and alder are better suited for constructing lodges. These materials are more readily available in wooded areas than other environments, such as bodies of water, British Columbia, or zoos. Moreover, beavers often have a preference for building their homes near or within the wooded areas they inhabit, as this allows for more efficient access to materials and provides a more natural and familiar environment for the beavers. In conclusion, wooded areas serve as the most suitable choice for beavers' construction supplies due to the abundance of essential materials, the beavers' preference for building near their habitats, and the specific requirements of their construction activities.",The answer is body of water,The answer is body of water,"This choice aligns with the beaver's ability to build dams and dams, which are essential for their survival and survival.","The beaver's building prowess comes from the presence of water, which provides them with the necessary materials for building. This choice aligns with the beaver's natural habitat and provides them with the necessary resources for building."
1,"[predict] Question: A car was hailed to chauffeur someone to the opera house, where was it heading? Choices: (a) go downtown (b) appear suddenly (c) go fast (d) bottom out (e) east",The answer is go downtown,"A car was hailed to chauffeur someone to the opera house, where was it heading? A car was hailed to chauffeur someone to the opera house, where was it heading?","Based on the context of a car being hailed to chauffeur someone to the opera house, it is likely that the destination is a downtown location. Opera houses are typically found in urban areas and downtown districts, making it a strong assumption that the car is headed to a downtown area.",The answer is go downtown,The answer is go downtown,"This choice aligns with the context of the question, which suggests that the car was heading to the opera house, which is a popular destination for opera performances.","The car was hailed to chauffeur someone to the opera house, which is a popular opera house in New York City. This choice aligns with the context of the question, which is about a car being hailed to chauffeur someone to an opera house. The other options do not accurately reflect the context of the question, such as (a) appear suddenly, (b) go fast, and (c) bottom out."
2,"[predict] Question: A child wants to play, what would they likely want? Choices: (a) fall down (b) breathe (c) play tag (d) be dismembered by a chainsaw (e) become adult",The answer is play tag,"This choice aligns with the child's desire to have fun and engage in physical activity. Tag is a classic game that requires speed, agility, and strategy, which makes it a popular choice among children.","This aligns with the child's desire to engage in fun and physical activities that promote development and well-being. Tag is a popular choice among children because it encourages physical activity, social interaction, and cognitive development. Compared to 

In [9]:
merged.shape

(1221, 8)

In [10]:
# Get rows where predictions differ between unrevised and revised versions
diff_df = merged[merged['Answers (student MT+CF:Unrevised)'] != merged['Answers (student MT+CF:Revised)']]
diff_df.sample(10)

,Prompt,Ground truth,Unrevised rationale,Revised rationale,Answers (student MT+CF:Unrevised),Answers (student MT+CF:Revised),Explanation (student MT+CF:Unrevised),Explanation (student MT+CF:Revised)
354,"[predict] Question: The weasel ran up away from danger, somebody joked only our first president could get him down from the what? Choices: (a) washington dc (b) ladder (c) natural history museum (d) cherry tree (e) chicken coop",The answer is cherry tree,"This choice aligns with the weasel escaping up away from danger, and somebody joking that only the first president could get him down from the cherry tree. This suggests that the tree was a cherry tree and that it was in some sort of danger, and the weasel's escape was a joke about how only","This aligns with the weasel escaping upward from danger, and the joke about only the first president being able to bring the weasel down from the tree. The choice is clear and concise and differentiates from the other options.",The answer is cherry tree,The answer is ladder,"This choice aligns with the weasel's run away from danger, suggesting that only our first president could get him down from the cherry tree.","The weasel ran up a ladder to escape danger, suggesting that only President George Washington could get him down from the tree. This choice aligns with the context of the question, which suggests that the weasel ran up a ladder to escape danger. The other options do not align with the context of the question, such as (a) natural history museum, (b) cherry tree, or (c) chicken coop, which do not relate to the weasel's run up from danger. (d) Chicken coop: Chicken coops are not suitable for the weasel's run up from danger, and (e) chicken coop: Chicken coops are not suitable for the weasel's run up from danger, and (e) chicken coop: Chicken coops are not suitable for the weasel's run up from danger, and (e) chicken coop: Chicken coops are not suitable for the weasel's run up from danger, and (e) chicken coops are not suitable for the weasel's run up from danger."
617,[predict] Question: Sally was afraid of danger and always double checked what? Choices: (a) fight enemy (b) secure (c) being safe (d) safety (e) vicinity,The answer is safety,"Sally's fear of danger suggests that she has a cautious and preventive approach towards ensuring her own safety and security. This is consistent with the definition of safety, which refers to the state of being free from harm or risk, or the measures taken to ensure it.","Sally's fear of danger suggests that she values her safety and takes preventive measures to ensure it. To clarify why safety is the correct answer: (a) fight enemy: Sally is not looking to engage in a fight, but rather to ensure her safety, (b) secure: Sally's fear of danger implies that she wants to secure her safety, (c) being safe: Sally's primary concern is her own safety, (d) safety: This option directly aligns with Sally's fear of danger and her desire to ensure her own safety.",The answer is being safe,The answer is safety,This choice aligns with Sally's fear of danger and her need to ensure that she is safe and secure. This choice aligns with her fear of danger and her need to ensure that she is safe and secure.,"The phrase ""sally was afraid of danger"" implies that Sally was concerned about the safety of herself and others, which aligns with the definition of safety as the ability to protect oneself from harm or harm. This choice aligns with the definition of safety as the ability to protect oneself from harm or harm."
1047,[predict] Question: Where is a dining area likely to be small? Choices: (a) cruise ship (b) home (c) mall (d) restaurant (e) dark cave,The answer is home,"It is where you would expect to find a small dining area, as it is usually located in a smaller space within a house, such as a kitchen or a breakfast nook, compared to larger dining areas found in restaurants or cruise ships.","A home is likely to have a small dining area due to spatial constr

In [11]:
previously_correct = diff_df["Ground truth"] == diff_df["Answers (student MT+CF:Unrevised)"]
afterwards_false   = diff_df["Ground truth"] != diff_df["Answers (student MT+CF:Revised)"]

In [12]:
correct_to_false = diff_df[(previously_correct & afterwards_false)]

In [13]:
false_to_correct = diff_df[(~previously_correct & ~afterwards_false)]

In [14]:
correct_to_false.sample(5)


,Prompt,Ground truth,Unrevised rationale,Revised rationale,Answers (student MT+CF:Unrevised),Answers (student MT+CF:Revised),Explanation (student MT+CF:Unrevised),Explanation (student MT+CF:Revised)
168,"[predict] Question: If you want to learn about the world and understand the real reasons behind cultural norms and mores, you have achieved a sense of what? Choices: (a) enlightenment (b) open mind (c) confusion (d) smartness (e) anger",The answer is enlightenment,"To be enlightened means to gain knowledge or understanding of a particular subject or issue, and in this context, it refers to learning about the world and understanding the real reasons behind cultural norms and mores. It suggests a deep and comprehensive understanding of a complex subject.","This choice reflects a deep understanding of the real reasons behind cultural norms and mores. To be enlightened means gaining knowledge and comprehension of complex issues, and in this context, it suggests a profound grasp of the world's cultural diversity and the underlying factors that shape these norms and mores. Confusion, smartness, and anger are not suitable choices for understanding cultural norms and mores because they do not connote a deep understanding of the subject. Instead, they imply a lack of comprehension, superficiality, or a negative emotional state. In summary, the correct answer is (a) enlightenment.",The answer is enlightenment,The answer is open mind,"This choice aligns with the idea that a sense of enlightenment is a sense of understanding and appreciation of the world around us, which can lead to a deeper understanding of the world and its people.","The phrase ""open mind"" implies a willingness to explore and learn about the world and its cultural norms and mores, which can lead to a sense of understanding and appreciation for the differences between cultures and societies. This choice aligns with the definition of enlightenment as a state of being able to see the world through a different perspective."
1027,[predict] Question: Where do you go on a night out before going to the bar? Choices: (a) new york city (b) las vegas (c) restaurant (d) nightclub (e) park,The answer is restaurant,It is a common practice to visit a restaurant before going to a bar or club. The restaurant serves as a pre-bar or pre-club venue where people can enjoy a meal or drinks with friends or family before heading to the bar or club for the night.,"Restaurants are popular pre-bar or pre-club destinations because they offer a variety of food and drink options in a social setting. Going to a restaurant before a night out allows people to enjoy a meal or drinks with friends or family, which can enhance the overall experience and create a more relaxed atmosphere before heading to the bar or club. This choice is more practical than options like Las Vegas or New York City, as it provides a more focused social experience and avoids the hassle of traveling far. Additionally, if you have specific dietary restrictions or preferences, going to a restaurant can be a better choice than ordering takeout or eating at home, as restaurants typically offer a wider range of options and cater to different tastes.",The answer is restaurant,The answer is nightclub,"This choice aligns with the typical nightlife experience of going to a restaurant before going to the bar, as restaurants typically serve food and drinks, making them a popular choice for those looking to enjoy a night out.","The nightclub is the most appropriate choice for a night out before going to the bar because it offers a variety of entertainment options, including live music, dancing, and other entertainment options. This choice aligns with the definition of a nightclub as a place where people go to socialize and enjoy a night out."
413,[predict] Question: A crane uses many a steel cable when working a what? Choices: (a) abaft (b) ship (c) winch (d) construction site (e) building,The answer is construction site,"Cranes are essent

In [15]:
import pandas as pd
import re

def extract_choices(prompt):
    matches = re.findall(r'\([a-e]\)\s*([^()]+?)(?=\s*\(|$)', prompt, flags=re.IGNORECASE)
    return [ans.strip() for ans in matches]

def count_mentions(text, choices, correct_answer):
    return [text.lower().count(ans.lower()) for ans in choices if ans != correct_answer]

def compare_mentions(df):
    rows = []
    for idx, row in df.iterrows():
        choices = extract_choices(row['Prompt'])
        correct = row['Ground truth'].replace('The answer is ', '').strip()
        unrevised = count_mentions(row['Unrevised rationale'], choices, correct)
        revised = count_mentions(row['Revised rationale'], choices, correct)
        wrong_mentioned_more = sum(r > u for u, r in zip(unrevised, revised))
        rows.append({
            'id': idx,
            'correct': correct,
            'choices': choices,
            'wrong_mentioned_more': wrong_mentioned_more
        })
    return pd.DataFrame(rows)

# usage:
correct_to_false_wrong_mentions = compare_mentions(correct_to_false)
false_to_correct_wrong_mentions = compare_mentions(false_to_correct)
# print(result)


In [16]:
#TODO should look at training data
correct_to_false_wrong_mentions['wrong_mentioned_more'].mean()

np.float64(1.7321428571428572)

In [17]:
false_to_correct_wrong_mentions['wrong_mentioned_more'].mean()

np.float64(1.4482758620689655)

## Use training data

In [18]:
trn_pth = Path("../../llm_outputs/consolidated/cqa-llama-train")

In [19]:
!ls $trn_pth

shard_0 shard_1 shard_2 shard_3 shard_4 shard_5 shard_6 shard_7 shard_8 shard_9


In [20]:
import sys

sys.path.append("../../src")

In [21]:
from utils import read_dataset

/Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
trn_data = read_dataset(trn_pth).to_pandas()

In [23]:
trn_data.columns

Index(['id', 'question', 'choices', 'answer',
       'counterfactual_prompt_answer_1_rationale',
       'counterfactual_prompt_answer_2_rationale',
       'counterfactual_prompt_answer_3_rationale',
       'counterfactual_prompt_answer_4_rationale',
       'counterfactual_prompt_answer_5_rationale',
       'few_shot_positive_rationale', 'few_shot_negative_rationale',
       'few_shot_revision'],
      dtype='object')

In [24]:
rename = {
    'question': 'Question',
    'choices': 'Choices',
    'answer': 'Ground truth',
    'few_shot_positive_rationale': 'Unrevised rationale',
    'few_shot_revision': 'Revised rationale',
    'answers_preds_unrevised': 'Answers (student MT+CF:Unrevised)',
    'answers_preds_revised': 'Answers (student MT+CF:Revised)',
    'rationales_preds_unrevised': 'Explanation (student MT+CF:Unrevised)',
    'rationales_preds_revised': 'Explanation (student MT+CF:Revised)'
    }

In [25]:
trn_data = trn_data.rename(rename, axis=1)

In [26]:
trn_data = trn_data.drop([col for col in trn_data.columns if not col in rename.values()], axis=1)

In [27]:
trn_data.columns    

Index(['Question', 'Choices', 'Ground truth', 'Unrevised rationale',
       'Revised rationale'],
      dtype='object')

In [28]:
trn_data.iloc[0]

Question                                                                                                                                                                                                                                                "There are 10 apples on an apple tree.  Three fall off.  Now there are X apples."  What is this an example of?
Choices                                                                                                                                                                                                                                                                                                    [park, coloring book, garden center, math problem, gravity]
Ground truth                                                                                                                                                                                                                                                                              

In [29]:
import pandas as pd

def count_mentions(text, choices, correct_answer):
    return [text.lower().count(ans.lower()) for ans in choices if ans != correct_answer]

def compare_mentions(df):
    """
    Compare mentions of incorrect choices between unrevised and revised rationales.
    
    Args:
        df (pandas.DataFrame): DataFrame containing 'Choices', 'Ground truth', 
            'Unrevised rationale', and 'Revised rationale' columns
    
    Returns:
        pandas.DataFrame: DataFrame with columns 'id', 'correct', 'choices', 
            'wrong_mentioned_more' containing analysis results
    """
    rows = []
    for idx, row in df.iterrows():
        choices = row['Choices']
        correct = row['Ground truth'].strip()
        unrevised = count_mentions(row['Unrevised rationale'], choices, correct)
        revised = count_mentions(row['Revised rationale'], choices, correct)
        wrong_mentioned_more = sum(r > u for u, r in zip(unrevised, revised))
        rows.append({
            'id': idx,
            'correct': correct,
            'choices': choices,
            'wrong_mentioned_more': wrong_mentioned_more
        })
    return pd.DataFrame(rows)

results = compare_mentions(trn_data)

In [30]:
results['wrong_mentioned_more'].mean()

np.float64(1.6628682886767272)

In [31]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# load a pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
#TODO add to requirements.txt sentence-transformers


/Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/danielhendriks/miniforge3/envs/model-distillation/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/danielhendriks/miniforge3/envs/model-distillation/bin/../lib/libjpeg.9

In [32]:
total_params = 0
for p in model.parameters():
    try:
        params = p.shape[0] * p.shape[1]
    except IndexError:
        params = p.shape[0]
    total_params += params
    
print(total_params // 1e6, "M")

22.0 M


In [33]:
from tqdm.auto import tqdm
def compute_similarity(df):
    sims = []
    for idx, row in tqdm(df.iterrows(), ncols=100):
        choices = [c for c in row['Choices'] if c != row['Ground truth']]
        unrevised_emb = model.encode(row['Unrevised rationale'], convert_to_tensor=True)
        revised_emb   = model.encode(row['Revised rationale'], convert_to_tensor=True)
        choice_embs   = model.encode(choices, convert_to_tensor=True)
        truth_embs    = model.encode(row['Ground truth'], convert_to_tensor=True)
        
        # compute cosine similarity to each distractor
        unrevised_sims_to_choices = util.cos_sim(unrevised_emb, choice_embs).squeeze().tolist()
        revised_sims_to_choices   = util.cos_sim(revised_emb, choice_embs).squeeze().tolist()
        
        unrevised_sims_to_answer = [util.cos_sim(unrevised_emb, truth_embs).squeeze().tolist()]
        revised_sims_to_answer   = [util.cos_sim(revised_emb, truth_embs).squeeze().tolist()]
        
        # compute average similarity to distractors
        sims.append({
            'id': idx,
            'avg_sim_unrevised_to_choices': sum(unrevised_sims_to_choices)/len(unrevised_sims_to_choices),
            'avg_sim_revised_to_choices': sum(revised_sims_to_choices)/len(revised_sims_to_choices),
            'delta_sim_to_choices': sum(revised_sims_to_choices)/len(revised_sims_to_choices) - sum(unrevised_sims_to_choices)/len(unrevised_sims_to_choices),
            'avg_sim_unrevised_to_answer': sum(unrevised_sims_to_answer)/len(unrevised_sims_to_answer),
            'avg_sim_revised_to_answer': sum(revised_sims_to_answer)/len(revised_sims_to_answer),
            'delta_sim_to_answer': sum(revised_sims_to_answer)/len(revised_sims_to_answer) - sum(unrevised_sims_to_answer)/len(unrevised_sims_to_answer)
        })
    return pd.DataFrame(sims)

# usage
sim_df = compute_similarity(trn_data.head(100))
print(sim_df.head())


100it [00:10,  9.25it/s]

   id  avg_sim_unrevised_to_choices  avg_sim_revised_to_choices  \
0   0                      0.048680                    0.212555   
1   1                      0.120945                    0.232977   
2   2                      0.217777                    0.286960   
3   3                      0.070505                    0.053296   
4   4                      0.238088                    0.218102   

   delta_sim_to_choices  avg_sim_unrevised_to_answer  \
0              0.163875                     0.452747   
1              0.112032                     0.407820   
2              0.069183                     0.557438   
3             -0.017209                     0.399255   
4             -0.019986                     0.614546   

   avg_sim_revised_to_answer  delta_sim_to_answer  
0                   0.507475             0.054729  
1                   0.381433            -0.026387  
2                   0.428989            -0.128449  
3                   0.321962            -0.077294  


In [34]:
sim_df.mean(axis=0)

id                              49.500000
avg_sim_unrevised_to_choices     0.147368
avg_sim_revised_to_choices       0.195507
delta_sim_to_choices             0.048138
avg_sim_unrevised_to_answer      0.417531
avg_sim_revised_to_answer        0.441186
delta_sim_to_answer              0.023655
dtype: float64

In [35]:
def compute_similarity_with_ground_truth(df):
    rows = []
    for idx, row in df.iterrows():
        choices = [c for c in row['Choices'] if c != row['Ground truth']]
        ground = row['Ground truth']
        
        unrevised_emb = model.encode(row['Unrevised rationale'], convert_to_tensor=True)
        revised_emb   = model.encode(row['Revised rationale'], convert_to_tensor=True)
        choice_embs   = model.encode(choices, convert_to_tensor=True)
        ground_emb    = model.encode(ground, convert_to_tensor=True)
        
        unrevised_sim_choices = util.cos_sim(unrevised_emb, choice_embs).squeeze().tolist()
        revised_sim_choices   = util.cos_sim(revised_emb, choice_embs).squeeze().tolist()
        
        unrevised_sim_ground = util.cos_sim(unrevised_emb, ground_emb).item()
        revised_sim_ground   = util.cos_sim(revised_emb, ground_emb).item()
        
        rows.append({
            'id': idx,
            'avg_sim_choices_unrevised': sum(unrevised_sim_choices)/len(unrevised_sim_choices),
            'avg_sim_choices_revised': sum(revised_sim_choices)/len(revised_sim_choices),
            'sim_ground_unrevised': unrevised_sim_ground,
            'sim_ground_revised': revised_sim_ground,
            'delta_choices': sum(revised_sim_choices)/len(revised_sim_choices) - sum(unrevised_sim_choices)/len(unrevised_sim_choices),
            'delta_ground': revised_sim_ground - unrevised_sim_ground
        })
    return pd.DataFrame(rows)

# usage
sim_df = compute_similarity_with_ground_truth(trn_data.head(100))
# print(sim_df.head())


In [36]:
sim_df.mean(axis=0)

id                           49.500000
avg_sim_choices_unrevised     0.147368
avg_sim_choices_revised       0.195507
sim_ground_unrevised          0.417531
sim_ground_revised            0.441186
delta_choices                 0.048138
delta_ground                  0.023655
dtype: float64

In [ ]:
import pandas as pd
import re
import spacy
from sentence_transformers import SentenceTransformer, util

# Load NLP models
nlp = spacy.load("en_core_web_sm")
# model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Basic text features ---
def length_features(text):
    doc = nlp(text)
    return {
        'num_words': len(doc),
        'num_sents': len(list(doc.sents)),
        'num_negations': sum(1 for token in doc if token.dep_ == 'neg')
    }

def count_distractor_mentions(text, choices, correct):
    return sum(text.lower().count(c.lower()) for c in choices if c != correct)

def semantic_features(text, choices, correct):
    text_emb = model.encode(text, convert_to_tensor=True)
    choice_embs = model.encode([c for c in choices if c != correct], convert_to_tensor=True)
    correct_emb = model.encode(correct, convert_to_tensor=True)
    
    avg_choice_sim = util.cos_sim(text_emb, choice_embs).mean().item()
    correct_sim = util.cos_sim(text_emb, correct_emb).item()
    return avg_choice_sim, correct_sim

# --- Main feature extraction ---
def extract_features(df):
    rows = []
    for idx, row in df.iterrows():
        choices = row['Choices']
        correct = row['Ground truth'].strip()
        unrevised_text = row['Unrevised rationale']
        revised_text = row['Revised rationale']
        
        # Length and negations
        len_unrev = length_features(unrevised_text)
        len_rev   = length_features(revised_text)
        
        # Distractor mentions
        dm_unrev = count_distractor_mentions(unrevised_text, choices, correct)
        dm_rev   = count_distractor_mentions(revised_text, choices, correct)
        
        # Semantic similarity
        avg_sim_unrev, sim_gt_unrev = semantic_features(unrevised_text, choices, correct)
        avg_sim_rev, sim_gt_rev     = semantic_features(revised_text, choices, correct)
        
        rows.append({
            'id': idx,
            'num_words_unrev': len_unrev['num_words'],
            'num_words_rev': len_rev['num_words'],
            'num_sents_unrev': len_unrev['num_sents'],
            'num_sents_rev': len_rev['num_sents'],
            'num_negations_unrev': len_unrev['num_negations'],
            'num_negations_rev': len_rev['num_negations'],
            'distractor_mentions_unrev': dm_unrev,
            'distractor_mentions_rev': dm_rev,
            'delta_distractor_mentions': dm_rev - dm_unrev,
            'avg_choice_sim_unrev': avg_sim_unrev,
            'avg_choice_sim_rev': avg_sim_rev,
            'delta_choice_sim': avg_sim_rev - avg_sim_unrev,
            'sim_ground_unrev': sim_gt_unrev,
            'sim_ground_rev': sim_gt_rev,
            'delta_ground_sim': sim_gt_rev - sim_gt_unrev
        })
    return pd.DataFrame(rows)

# Usage:
feature_df = extract_features(trn_data)
# print(feature_df.head())
#TODO run analysis with full dataset


In [ ]:
feature_df.mean(axis=0)

id                           4870.000000
num_words_unrev                44.544297
num_words_rev                 114.299148
num_sents_unrev                 1.717791
num_sents_rev                   4.506108
num_negations_unrev             0.129966
num_negations_rev               1.256237
distractor_mentions_unrev       0.181809
distractor_mentions_rev         2.205934
delta_distractor_mentions       2.024125
avg_choice_sim_unrev            0.167756
avg_choice_sim_rev              0.209466
delta_choice_sim                0.041710
sim_ground_unrev                0.424819
sim_ground_rev                  0.436203
delta_ground_sim                0.011385
dtype: float64

In [ ]:
len(trn_data)